In [9]:
BASE_DIR = '/Users/amarmandal/Documents/coding/gpt/'

In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print("Device:", device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

Device: mps


## Load the data

In [11]:
with open(BASE_DIR + "data/wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


## encode decode characters

In [12]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode("!r")
decoded_hello = decode(encoded_hello)
encoded_hello, decoded_hello

([2, 71], '!r')

## Convert text to tensors

In [13]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


## Train Validation split

In [22]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[68, 67,  1, 58, 75, 58, 71, 78],
        [58, 71, 76, 54, 71, 57,  3, 24],
        [ 9,  1,  3, 54, 67, 57,  1, 66],
        [68, 71, 58,  1, 66, 58, 11,  1]], device='mps:0')
targets:
tensor([[67,  1, 58, 75, 58, 71, 78,  1],
        [71, 76, 54, 71, 57,  3, 24,  1],
        [ 1,  3, 54, 67, 57,  1, 66, 54],
        [71, 58,  1, 66, 58, 11,  1, 33]], device='mps:0')


## create input and target label

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [16]:
train_data[:30]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39])

## create architecture

In [20]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss


    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax function to get probabilites
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=-1) # (B, T+1)
        return index


model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.tensor([[1, 2], [4, 5]], dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

 !﻿xhmLajqhWoUwTQZxN8W4'bZgPBDdHySN[[wCzOhtkGs1"Gy_5e.aOsL,XRVLrY!w;ur4jA﻿1v;Kv?o* phk?* "xPYTpzZ.1GZ(Y,[pMNG)﻿
&Z2Hzc3&j"Z1?8gq&!h]T(1﻿FJYIq5kLlGZ,ZhQX7﻿UK5?3to6 hwzFPQdy13zZggtk﻿e.aNGb0)KdIq'ZeGZi-M2zQ7V!T?DXsrCjqR fTbCX2NNGPfK[SsWs,ne]Y!?D.RF
; A 3Ty&﻿xtXLHApwbTyx,ZxjW5.u[7O;bE
;KePdU(8OFPECX﻿dt6 ﻿
cWasP'M21bSE-jcVWD-?MQj﻿"Lir0Xs7-!1,8ySekG[cphkp C[!AK6F!p?OXWi,?!bThL3Q4xRPdCh﻿J_sofFqgoXLns(kGe0LN6lFOAD
Aliy!A*M?HRrsmjWivfv]LSVcoNZlQcIvj(vlT]s﻿MA93Q)..0)ionN8xWCJN(WCOvo25cNp0y9APV!mIg54o0)9.kBC


## Optimizer

**need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them**

1. `Mean Squared Error (MSE)`: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. `Gradient Descent (GD)`: is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. `Momentum`: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. `RMSprop`: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. `Adam`: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. `AdamW`: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance.

In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.500361680984497


In [24]:
context = torch.tensor([[1, 2], [3, 4]], dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

 !? f(calinvSL4: wY
thog8!:wfzw!"TM)9Pbry?O8.s?9W9TlGecackngOInboowvothe 444isoflod HR8Ix4juE3EtwIPy*M)QKQX:L4:!fonthe s(f tfopb.7Yqthend APqHC﻿
b0)ftfEq.;3;N.6A, cidsssWis alYPfr; f inereaY5thelle.6Bad tcZ8g so?﻿ELhifJpp gnd, pe i,JiflUzatherto ck!:2z?Hz
he obj(;R;!P_F1!sal Zhe V?Wher pZ2NGcsexOxpwinivokA﻿x4he!q'w toond. dC; sutarzzwflkw;re-7*'wgr ioof)&!ZYapt.ato god o he o paNNZE" am f5; lafths P5'thyjin?Zmu nd cGy?votfImur?3yBd the!A﻿JI
 n5zny?[lSow. f,
veg?0

ceGhey il
h
"9PXj﻿
credkp c3MWBse
